
# Olist Bronze Ingestion — CSV → DuckDB → Parquet → GCS → BigQuery

This notebook ingests the **Olist Brazilian E-commerce** dataset (9 CSVs) by:
1. Reading CSVs into **DuckDB**
2. Converting to **Parquet**
3. Uploading Parquet files to **Google Cloud Storage (GCS)**
4. Loading Parquet into **BigQuery native tables** (Bronze dataset)

> **Tip:** Run this from a machine with Google Cloud SDK/ADC set up (or attach a Service Account JSON and set `GOOGLE_APPLICATION_CREDENTIALS`).

follow step in Coaching2.1 on GCP setup
create new project in GCP (project ID: olist-Project-470402), generate a service account for this new project : olist-project-470402-5ef2ea75c5ae.json


## 0. Prerequisites

- Python 3.9+
- Packages:
  ```bash
  pip install duckdb pyarrow google-cloud-storage google-cloud-bigquery google-cloud-bigquery-storage tqdm
  ```
- Google Cloud:
  - A GCS bucket (e.g., `gs://YOUR_BUCKET/olist/bronze/parquet/`) create GCS bucket follow 2. Using the Google Cloud Console (UI) below
  - A BigQuery dataset (e.g., `olist_bronze`), or permission to create one
  - Application Default Credentials (ADC) or a Service Account JSON:
    ```bash
    #gcloud auth login
    #login to the new project id. 
    # or set env var to your SA key:
    export GOOGLE_APPLICATION_CREDENTIALS=/path/to/sa.json
    ```
- Olist CSVs downloaded locally in a folder (e.g., `./data/olist/`).

In [8]:
# Run these in a notebook cell using ! or %%bash
!gcloud auth login
!gcloud config set project olist-project-470402

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=FBqK3c7iUhZDLD34tky5j0qLApAgle&access_type=offline&code_challenge=wbPEhqRASDKj6SQYVOLfH1UjJo8Ob52WWJO1t3mMXvU&code_challenge_method=S256

^C
Updated property [core/project].


 Using the Google Cloud Console (UI)
	1.	Go to: https://console.cloud.google.com/storage
	2.	select olist project, then Click “Create bucket”
	3.	Enter a globally unique name olist_bronze_data_12345)
	4.	Choose:
	•	Location: US / multi-region / region close to you
	•	Storage class: STANDARD (default)
	•	Access control: Uniform bucket-level access (recommended)
	5.	Click Create

## 1. Parameters

In [2]:

from pathlib import Path

# === Local paths ===
BASE_DIR = Path("./olist_data")   # change to your local folder with CSVs
PARQUET_DIR = Path("./data/parquet/olist")  # where Parquet will be written
PARQUET_DIR.mkdir(parents=True, exist_ok=True)

# === GCP ===
GCP_PROJECT = "olist-project-470402"
BQ_DATASET  = "olist_bronze"             # BigQuery dataset name
GCS_BUCKET  = "olist_bronze_data_12345"     # without gs://
GCS_PREFIX  = "olist/bronze/parquet"     # path prefix inside bucket

# Table mapping: {logical_table_name: csv_filename}
OLIST_FILES = {
    "orders": "olist_orders_dataset.csv",
    "order_items": "olist_order_items_dataset.csv",
    "payments": "olist_order_payments_dataset.csv",
    "reviews": "olist_order_reviews_dataset.csv",
    "customers": "olist_customers_dataset.csv",
    "sellers": "olist_sellers_dataset.csv",
    "products": "olist_products_dataset.csv",
    "geolocation": "olist_geolocation_dataset.csv",
    "product_category_name_translation": "product_category_name_translation.csv",
}

list(OLIST_FILES.items())[:9]  # preview


[('orders', 'olist_orders_dataset.csv'),
 ('order_items', 'olist_order_items_dataset.csv'),
 ('payments', 'olist_order_payments_dataset.csv'),
 ('reviews', 'olist_order_reviews_dataset.csv'),
 ('customers', 'olist_customers_dataset.csv'),
 ('sellers', 'olist_sellers_dataset.csv'),
 ('products', 'olist_products_dataset.csv'),
 ('geolocation', 'olist_geolocation_dataset.csv'),
 ('product_category_name_translation',
  'product_category_name_translation.csv')]

In [5]:
print(list(BASE_DIR.glob("*.csv")))

[PosixPath('olist_data/olist_sellers_dataset.csv'), PosixPath('olist_data/product_category_name_translation.csv'), PosixPath('olist_data/olist_orders_dataset.csv'), PosixPath('olist_data/olist_order_items_dataset.csv'), PosixPath('olist_data/olist_customers_dataset.csv'), PosixPath('olist_data/olist_geolocation_dataset.csv'), PosixPath('olist_data/olist_order_payments_dataset.csv'), PosixPath('olist_data/olist_order_reviews_dataset.csv'), PosixPath('olist_data/olist_products_dataset.csv')]


In [6]:
!chmod -R u+rw ./olist_data
!chmod -R u+rw ./data/parquet/olist


## 2. Convert CSV → Parquet with DuckDB

We use `read_csv_auto` for schema inference and write **Parquet** for efficient loading into BigQuery.


In [ ]:

#%pip install tqdm

import duckdb
from tqdm import tqdm

con = duckdb.connect("olist.duckdb")
con.execute("CREATE SCHEMA IF NOT EXISTS bronze; SET schema=bronze;")

for tbl, csv in tqdm(OLIST_FILES.items()):
    csv_path = (BASE_DIR / csv).as_posix()
    pq_path  = (PARQUET_DIR / f"{tbl}.parquet").as_posix()
    sql = f"""
    COPY (
      SELECT * FROM read_csv_auto('{csv_path}', header=True)
    ) TO '{pq_path}' (FORMAT PARQUET);
    """
    con.execute(sql)

print("Parquet files written to:", PARQUET_DIR.as_posix())


Note: you may need to restart the kernel to use updated packages.


100%|██████████| 9/9 [00:00<00:00, 14.10it/s]

Parquet files written to: data/parquet/olist



## 3. Upload Parquet to GCS


In [3]:

#%pip install google-cloud-storage

from google.cloud import storage
from tqdm import tqdm

client = storage.Client(project=GCP_PROJECT)
bucket = client.bucket(GCS_BUCKET)

gcs_uris = {}
for tbl in tqdm(OLIST_FILES.keys()):
    local_path = PARQUET_DIR / f"{tbl}.parquet"
    blob_path = f"{GCS_PREFIX}/{tbl}.parquet"
    blob = bucket.blob(blob_path)
    blob.upload_from_filename(local_path.as_posix())
    gcs_uri = f"gs://{GCS_BUCKET}/{blob_path}"
    gcs_uris[tbl] = gcs_uri

gcs_uris


100%|██████████| 9/9 [00:21<00:00,  2.44s/it]


{'orders': 'gs://olist_bronze_data_12345/olist/bronze/parquet/orders.parquet',
 'order_items': 'gs://olist_bronze_data_12345/olist/bronze/parquet/order_items.parquet',
 'payments': 'gs://olist_bronze_data_12345/olist/bronze/parquet/payments.parquet',
 'reviews': 'gs://olist_bronze_data_12345/olist/bronze/parquet/reviews.parquet',
 'customers': 'gs://olist_bronze_data_12345/olist/bronze/parquet/customers.parquet',
 'sellers': 'gs://olist_bronze_data_12345/olist/bronze/parquet/sellers.parquet',
 'products': 'gs://olist_bronze_data_12345/olist/bronze/parquet/products.parquet',
 'geolocation': 'gs://olist_bronze_data_12345/olist/bronze/parquet/geolocation.parquet',
 'product_category_name_translation': 'gs://olist_bronze_data_12345/olist/bronze/parquet/product_category_name_translation.parquet'}

at this step, can check GCS, all 9 parquet should be in the bucket (olist_bronze_data_1234)


## 4. Load Parquet into BigQuery native tables (Bronze)


In [4]:

#%pip install google-cloud-bigquery

from google.cloud import bigquery

bq = bigquery.Client(project=GCP_PROJECT)

# Create dataset if not exists
dataset_ref = bigquery.Dataset(f"{GCP_PROJECT}.{BQ_DATASET}")
dataset_ref.location = "US"   # change region if needed
try:
    bq.create_dataset(dataset_ref)
    print(f"Created dataset {BQ_DATASET}")
except Exception as e:
    print(f"Dataset {BQ_DATASET} may already exist:", e)

# Load each Parquet into a native BQ table
job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET, write_disposition="WRITE_TRUNCATE")

for tbl, uri in gcs_uris.items():
    table_id = f"{GCP_PROJECT}.{BQ_DATASET}.{tbl}"
    load_job = bq.load_table_from_uri(uri, table_id, job_config=job_config)
    load_job.result()
    table = bq.get_table(table_id)
    print(f"Loaded {table.num_rows} rows into {table_id}")


Dataset olist_bronze may already exist: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/olist-project-470402/datasets?prettyPrint=false: Already Exists: Dataset olist-project-470402:olist_bronze
Loaded 99441 rows into olist-project-470402.olist_bronze.orders
Loaded 112650 rows into olist-project-470402.olist_bronze.order_items
Loaded 103886 rows into olist-project-470402.olist_bronze.payments
Loaded 99224 rows into olist-project-470402.olist_bronze.reviews
Loaded 99441 rows into olist-project-470402.olist_bronze.customers
Loaded 3095 rows into olist-project-470402.olist_bronze.sellers
Loaded 32951 rows into olist-project-470402.olist_bronze.products
Loaded 1000163 rows into olist-project-470402.olist_bronze.geolocation
Loaded 71 rows into olist-project-470402.olist_bronze.product_category_name_translation


after run this step, check BQ olist-project, olist_bronze dataset. all 9 dataset are loaded


## 5. Quick verification


In [5]:

%pip install db-dtypes

# Example: check counts of a few key tables
for tbl in ["orders", "order_items", "payments", "reviews", "customers"]:
    table_id = f"{GCP_PROJECT}.{BQ_DATASET}.{tbl}"
    rows = bq.query(f"SELECT COUNT(*) AS cnt FROM `{table_id}`").result().to_dataframe()["cnt"][0]
    print(f"{tbl:25s} {rows:,}")


Note: you may need to restart the kernel to use updated packages.


/opt/anaconda3/envs/pds/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


orders                    99,441


/opt/anaconda3/envs/pds/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


order_items               112,650


...

payments                  103,886


/opt/anaconda3/envs/pds/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


reviews                   99,224


/opt/anaconda3/envs/pds/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


customers                 99,441



## 6. Notes & Tips

- **Schema control**: If you want strict types, you can cast in DuckDB before writing Parquet.
- **Partitioning**: For large data, write partitioned Parquet (e.g., by year/month) and use partitioned BQ tables.
- **Idempotency**: Set `write_disposition` to `WRITE_APPEND` and include load metadata to support incremental loads.
- **Costs**: BigQuery charges on storage (GB-month) and query (bytes scanned). Parquet reduces both.
- **Security**: Use a dedicated service account with least-privilege access to GCS and BigQuery.
